In [1]:
import os
import numpy as np
import random
import datetime
import subprocess
import shutil

In [2]:
#this function will return the length in seconds of a file when given the full file path
def get_length(filename):
    result = subprocess.run(["ffprobe", "-v", "error", "-show_entries",
                             "format=duration", "-of",
                             "default=noprint_wrappers=1:nokey=1", filename],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT)
    return round(float(result.stdout))

#this function will return a list of every file in a directory when given the path to a particular folder
def get_full_path(path):
    arr=[]
    for file in os.listdir(path):
        if not file.startswith('.'):
            location=path+file
            if os.path.isdir(location):
                continue
            else:
                arr.append(location)
    return np.array(arr)

#this function will return the lengths for a series of file when given a list of file paths
def length_vids(files):
    return np.array([get_length(entry) for entry in files])

def find(s, ch):
    return [i for i, ltr in enumerate(s) if ltr == ch]

In [3]:
months=np.array(['January','February','March','April','May','June','July','August','September',
       'October','November','December'])

mydate=datetime.datetime.now()
curmonth=mydate.strftime("%B")

In [4]:
#pull in and get the full paths of all the files in various folders
commercial_full_path=get_full_path('/Users/bwal/Desktop/Archived Bumps & Commercials/Commercials/FOX/')
promo_full_path=get_full_path('/Users/bwal/Desktop/Archived Bumps & Commercials/FOX/Promos/')
bump_full_path=get_full_path('/Users/bwal/Desktop/Archived Bumps & Commercials/FOX/Bumps/')

In [5]:
#remove commercials, bumps, promos that occur in different months than the current month

curmonth_promo_full_path=np.array([promo for promo in promo_full_path if curmonth in promo])
curmonth_commercial_full_path=np.array([promo for promo in commercial_full_path if curmonth in promo])
curmonth_bump_full_path=np.array([promo for promo in bump_full_path if curmonth in promo])

for month in months:
    promo_full_path=np.array([promo for promo in promo_full_path if month not in promo])
    commercial_full_path=np.array([promo for promo in commercial_full_path if month not in promo])
    bump_full_path=np.array([promo for promo in bump_full_path if month not in promo])
    
promo_full_path=np.append(promo_full_path,curmonth_promo_full_path)
commercial_full_path=np.append(commercial_full_path,curmonth_commercial_full_path)
bump_full_path=np.append(bump_full_path,curmonth_bump_full_path)

In [6]:
#get the lengths for all video files in each subdirectory from above
commercial_lengths=length_vids(commercial_full_path)
promo_lengths=length_vids(promo_full_path)
bump_lengths=length_vids(bump_full_path)

In [7]:
#establish starting probabilities. settled on an even 5% per category. can be modified by any value between 0 and 1
flag_dict={'promo':0.05}

In [8]:
#this function is responsible for generating commercial blocks
def commercial_generator(length,probs,master_order,
                        promo_lengths=promo_lengths,promo_full_path=promo_full_path,
                        commercial_full_path=commercial_full_path,
                        commercial_lengths=commercial_lengths):

    commercials_list=[]

    commercial_block_size=length

    #assign probality dictionary to a temp variable
    flag_dict=probs

    #while there's still at least 10 seconds left in the commercial block, continue to loop through and add more commercials
    while commercial_block_size>10:

        #calculate the remaining probability after subtracting the other probabilities
        remainder=1-(flag_dict['promo'])
        #assign all probabilities 
        probabilities=[flag_dict['promo'],remainder]
        #randomly select which 'type' of commercial will be next from these 4 specific options - this can be modified accordingly
        commercial_type=random.choices(['promo','commercial'],weights=probabilities)[0]

        #check the commercial type and cross reference the selected promo against the lengths calculated earlier
        if commercial_type=='promo':
            commercial=random.choice(promo_full_path)
            diff=int(promo_lengths[promo_full_path==commercial])
        elif commercial_type=='commercial':
            commercial=random.choice(commercial_full_path)
            diff=int(commercial_lengths[commercial_full_path==commercial])

        #subtract the remaining time left in the commercial block by the length of the commercial/promo just selected
        commercial_block_size=commercial_block_size-diff
        
        #if the addition of the new promo causes the commercial block to run 10 seconds over the specified time, then skip 
        #it and select another one
        if commercial_block_size<-10:
            commercial_block_size=commercial_block_size+diff
        #if the commercial has already been selected for the broadcast then skip it and select another one
        elif commercial in commercials_list:
            commercial_block_size=commercial_block_size+diff
        #if the commercial is part of the last 30 files in the order, toss it out for a new one to keep it fresh
        elif commercial in master_order[-30:]:
            commercial_block_size=commercial_block_size+diff

        #otherwise if the earlier 2 criteria aren't met, add the commercial to the list
        else:
            commercials_list.append(commercial)
            
            #if the selected commercial type is anything other than a normal commercial, reset the probability of selecting
            #that commercial type again to zero to avoid repeats
            if commercial_type!='commercial':
                flag_dict[commercial_type]=0

            #once a commercial is added to the list, adjust the probabilities so they tick upwards
            #ccf probabilities tick up at 1/4 the rate of the other promos just because - this can be adjusted based 
            #on personal preference
            for ctype in ['promo']:
                flag_dict[ctype]+=0.01
                
        #print(commercial_block_size,diff)
        #print(probabilities)

    #once done, this function will return a list of commercials, and the probabilities so that they can be carried
    #over as input for the next commercial break
    return(commercials_list,flag_dict)

In [9]:
#list of shows you want in block in order
shows=['Simpsons','King of the Hill','Malcom in the Middle','X Files']

master_order=[]

#what episode number you'd like to select for certain shows
enum_dict={'King of the Hill':0,'Malcom in the Middle':0,'X Files':0}


#start looping through each show in the list
for i,show in enumerate(shows):
    show_files=get_full_path('/Volumes/Elements/Broadcast_Shows/FOX/'+show+'/mp4/Subsections/')
    show_files.sort()

    #look through the array of shows in previous broadcasts and isolate episodes for current show
    master_path='/Users/bwal/Documents/Block Dump/past_eps_used/'
    with open(master_path+show+'.txt','r+') as f:
        f.seek(0)
        past_episodes_used=f.readlines()
    #for each episode of this show that's been used before, remove it from the selectable episodes list
    #because we don't want to repeat episodes
    if len(past_episodes_used)>0:
        if ((len(past_episodes_used)/len(show_files))>0.9) & (show not in enum_dict):
            with open('/Users/bwal/Documents/Block Dump/past_eps_used/'+show+'.txt','w') as f:
                f.seek(0)
        
        elif (show in enum_dict) & (show_files[-1] in past_episodes_used):
            with open('/Users/bwal/Documents/Block Dump/past_eps_used/'+show+'.txt','w') as f:
                f.seek(0)

        else:
            for past_episode in past_episodes_used:
                past_episode=past_episode.replace('E:','/Volumes/Elements')
                if past_episode[:-2] in show_files:
                    index=np.argwhere(show_files==past_episode[:-2])[0,0]
                    show_files=np.delete(show_files,index)  

    if curmonth!='October' and show=='Simpsons':
        show_files=show_files[np.array(['horror' not in show for show in show_files])==True]
    elif curmonth=='October' and show=='Simpsons':
        show_files=show_files[np.array(['horror' in show for show in show_files])==True]

    #if the show uses a specified episode num in the dict, use that episode num
    #otherwise select a random episode
    if show in enum_dict:
        show_selector=show_files[enum_dict[show]]
        show_segments=show_files[np.array([show_selector[:-8] in segment for segment in show_files])==True]
    else:
        ep_selector=random.choice(show_files)
        show_segments=show_files[np.array([ep_selector[:-8] in segment for segment in show_files])==True]
    
    show_fadeout_bumps=bump_full_path[np.array([show+' Fade Out' in bump for bump in bump_full_path])==True]

    for i,segment in enumerate(show_segments):
        #add the first part of the show to the block order
        master_order.append(segment)

        if segment!=show_segments[-1]:
            
            if len(show_fadeout_bumps)!=0 and segment==show_segments[0]:
                fadeout_bump=random.choice(show_fadeout_bumps)

                index=np.argwhere(show_fadeout_bumps==fadeout_bump)
                show_fadeout_bumps=np.delete(show_fadeout_bumps,index)

                master_order.append(fadeout_bump)
            
            #generate a commercial break using the function above, specificying the length of the block in seconds followed by
            #the probabilities and then append each entry in the commercial block to the master file
            tmp_cblock,flag_dict=commercial_generator(210,flag_dict,master_order)
            for tmp_c in tmp_cblock:
                master_order.append(tmp_c)
                
            #throw a FOX promo for the fade back in to the show - pretty typical for this block
            random_promo=random.choice(promo_full_path)
            while random_promo in master_order[-30:]:
                random_promo=random.choice(promo_full_path)
            master_order.append(random_promo)


        #if the current segment is the last of the show
        elif (segment==show_segments[-1]):

            if show!=shows[-1]:
                #put in some commercials between shows
                tmp_cblock,flag_dict=commercial_generator(90,flag_dict,master_order)
                for tmp_c in tmp_cblock:
                    master_order.append(tmp_c)
                
print('Done!')

Done!


In [10]:
master_order

['/Volumes/Elements/Broadcast_Shows/FOX/Simpsons/mp4/Subsections/Simpsons E186_000.mp4',
 '/Users/bwal/Desktop/Archived Bumps & Commercials/FOX/Bumps/Simpsons Fade Out Coca Cola.mp4',
 '/Users/bwal/Desktop/Archived Bumps & Commercials/Commercials/FOX/2000 Fujitsu Transform Yourself.mp4',
 '/Users/bwal/Desktop/Archived Bumps & Commercials/Commercials/FOX/2003 Qwest High Speed Internet.mp4',
 '/Users/bwal/Desktop/Archived Bumps & Commercials/Commercials/FOX/2000 ReplayTV.mp4',
 '/Users/bwal/Desktop/Archived Bumps & Commercials/Commercials/FOX/2000 Target Mr Clean Together.mp4',
 '/Users/bwal/Desktop/Archived Bumps & Commercials/Commercials/FOX/1998 Jeep Sport Utility.mp4',
 '/Users/bwal/Desktop/Archived Bumps & Commercials/Commercials/FOX/2001 1800 ATT Carrot Top Dogs on Leash.mp4',
 '/Users/bwal/Desktop/Archived Bumps & Commercials/Commercials/FOX/1999 Olive Garden Ordering the Best Dish.mp4',
 '/Users/bwal/Desktop/Archived Bumps & Commercials/Commercials/FOX/2000 A Century To Remember.

In [11]:
past_shows=np.array(master_order)[np.array(['Broadcast_Shows' in line for line in master_order])]

show_uqid=np.array([])

for i,entry in enumerate(past_shows):

    if i==0:
        show_uqid=np.append(show_uqid,shows[i])
        j=0
    else:
        if entry[:-5]==past_shows[i-1][:-5]:
            show_uqid=np.append(show_uqid,shows[j])
        else:
            j+=1
            show_uqid=np.append(show_uqid,shows[j])

master_path='/Users/bwal/Documents/Block Dump/past_eps_used/'

for i,uqid in enumerate(show_uqid):
    with open(master_path+uqid+'.txt','r+') as f:
        f.seek(0)
        contents=f.readlines()
        
    with open(master_path+uqid+'.txt','w+') as f:
        if past_shows[i][6:] not in contents:
            contents.insert(-1,past_shows[i]+"'\n")
        contents = "".join(contents)
        f.write(contents)

In [12]:
np.savetxt('/Users/bwal/Documents/Block Dump/Blocks/FOX_DECEMBER_005.txt',np.array(['file '+"'"+entry+"'" for entry in master_order]),fmt='%s')